In [27]:
import pandas as pd
import jpy_tools.parseSnake2 as jps

In [28]:
configPath = '/public/home/liuzj/scripts/pipeline/10xAnalysis/snakemake/config.yaml'
snakemakeFilePath = '/public/home/liuzj/scripts/pipeline/10xAnalysis/snakemake/snakefile'

In [29]:
snakeFile = jps.SnakeFile()

In [30]:
snakeHeader = jps.SnakeHeader(snakeFile, configPath)
snakeHeader.addCode("sampleLs = list(config['Samples'].keys())")
config = snakeHeader.getConfig()
snakeHeader

import pandas as pd
#configfile: "/public/home/liuzj/scripts/pipeline/10xAnalysis/snakemake/config.yaml"
pipelineDir = config['pipelineDir']
resultDir = config["resultDir"].rstrip("/") + "/"
pipelineDir = config["pipelineDir"].rstrip("/") + "/"
sampleLs = list(config['Samples'].keys())

In [31]:
config

{'cellRangerPath': '/public/home/liuzj/softwares/environment/cellranger6',
 'cellRangerIndex': '/public/home/liuzj/data/singleCellRef/mm10_UCSC_ensGene',
 'resultDir': '/public/home/liuzj/projects/singleCell/singleCellMouseNeuron/01_data/publicData/',
 'pipelineDir': '/public/home/liuzj/projects/singleCell/singleCellMouseNeuron/01_data/publicData/',
 'genomeBed': '/public/home/liuzj/data/mm10_UCSC/mm10.ensGene.noScaffold.gtf.changeTrsId.bed',
 'Samples': {'artis_A3': {'fastq': '/public/home/liuzj/projects/singleCell/singleCellMouseNeuron/01_data/publicData/SRR9895509_medialPrefrontalCortex/artis_A3',
   'includeIntron': True,
   'expectCell': 5000,
   'r2Length': 91},
  'artis_A4': {'fastq': '/public/home/liuzj/projects/singleCell/singleCellMouseNeuron/01_data/publicData/allen_biorxiv/Raw_data_scRNA_10X_v3_A',
   'includeIntron': True,
   'expectCell': 5000,
   'r2Length': 91}}}

In [32]:
pipelineDir = config['pipelineDir']
resultDir = config["resultDir"].rstrip("/") + "/"
pipelineDir = config["pipelineDir"].rstrip("/") + "/"
sampleLs = list(config['Samples'].keys())

In [33]:
cellRangerParamsDt = config["Samples"]
cellRangerParamsDf = pd.DataFrame(cellRangerParamsDt).T.pipe(
    lambda df: df.assign(id=df.index, outputDir=df.index)
)
cellRangerParamsDf = cellRangerParamsDf.pipe(
    lambda df: df.assign(
        bamFile = df.outputDir+ "/" + df.index  + "/outs/possorted_genome_bam.bam",
        barcodeFile = df.outputDir+ "/" + df.index  + "/outs/filtered_feature_bc_matrix/barcodes.tsv.gz",
        tenXResultDir = df.outputDir+ "/" + df.index
    )
)
cellRangerParamsDf["cellRangerIndex"] = config["cellRangerIndex"]
if 'sample' in cellRangerParamsDf.columns:
    pass
else:
    cellRangerParamsDf['sample'] = False
cellRangerParamsDf

,fastq,includeIntron,expectCell,r2Length,id,outputDir,bamFile,barcodeFile,tenXResultDir,cellRangerIndex,sample
artis_A3,/public/home/liuzj/projects/singleCell/singleC...,True,5000,91,artis_A3,artis_A3,artis_A3/artis_A3/outs/possorted_genome_bam.bam,artis_A3/artis_A3/outs/filtered_feature_bc_mat...,artis_A3/artis_A3,/public/home/liuzj/data/singleCellRef/mm10_UCS...,False
artis_A4,/public/home/liuzj/projects/singleCell/singleC...,True,5000,91,artis_A4,artis_A4,artis_A4/artis_A4/outs/possorted_genome_bam.bam,artis_A4/artis_A4/outs/filtered_feature_bc_mat...,artis_A4/artis_A4,/public/home/liuzj/data/singleCellRef/mm10_UCS...,False


In [34]:
cellRangerRule = jps.SnakeRule(snakeFile, 'cellRanger', 48)
cellRangerRule.addCode("""
cellRangerParamsDt = config["Samples"]
cellRangerParamsDf = pd.DataFrame(cellRangerParamsDt).T.pipe(
    lambda df: df.assign(id=df.index, outputDir=df.index)
)
cellRangerParamsDf = cellRangerParamsDf.pipe(
    lambda df: df.assign(
        bamFile = df.outputDir+ "/" + df.index  + "/outs/possorted_genome_bam.bam",
        barcodeFile = df.outputDir+ "/" + df.index  + "/outs/filtered_feature_bc_matrix/barcodes.tsv.gz",
        tenXResultDir = df.outputDir+ "/" + df.index
    )
)
cellRangerParamsDf["cellRangerIndex"] = config["cellRangerIndex"]
if 'sample' in cellRangerParamsDf.columns:
    pass
else:
    cellRangerParamsDf['sample'] = False
""")
cellRangerRule.addMetaDf('cellRangerParamsDf', ['outputDir', 'bamFile', 'barcodeFile', 'tenXResultDir'])
cellRangerRule.addMain('input', ['fastq'], )
cellRangerRule.addMain('params', ['includeIntron', 'expectCell', 'r2Length', 'id', 'outputDir','cellRangerIndex', 'sample'])
cellRangerRule.setShell("""
mkdir -p {params.outputDir}
cd {params.outputDir}
cmd="cellranger6 count --id {params.id} --transcriptome {params.cellRangerIndex} --fastqs {input.fastq} --expect-cells {params.expectCell} --localcores {threads} --r2-length {params.r2Length}"
if [ {params.includeIntron} = True ]
then
    cmd="${{cmd}} --include-introns"
fi
if [ {params.sample} != True ]
then
    cmd="${{cmd}} --sample {params.sample}"
fi
echo $cmd
eval $cmd
touch {output.cellRangerFinished}
""")
cellRangerRule

2022-10-19 16:26:13.798 | INFO     | jpy_tools.parseSnake2:addRule:55 - cellRanger step num: 1
2022-10-19 16:26:13.799 | WARNING  | jpy_tools.parseSnake2:addMetaDf:203 - please set `metaDf` if you want to record dataframe content in snakefile



## get parameter of rule `cellRanger` ##
cellRangerParamsDt = config["Samples"]
cellRangerParamsDf = pd.DataFrame(cellRangerParamsDt).T.pipe(
    lambda df: df.assign(id=df.index, outputDir=df.index)
)
cellRangerParamsDf = cellRangerParamsDf.pipe(
    lambda df: df.assign(
        bamFile = df.outputDir+ "/" + df.index  + "/outs/possorted_genome_bam.bam",
        barcodeFile = df.outputDir+ "/" + df.index  + "/outs/filtered_feature_bc_matrix/barcodes.tsv.gz",
        tenXResultDir = df.outputDir+ "/" + df.index
    )
)
cellRangerParamsDf["cellRangerIndex"] = config["cellRangerIndex"]
if 'sample' in cellRangerParamsDf.columns:
    pass
else:
    cellRangerParamsDf['sample'] = False
for column in ['outputDir', 'bamFile', 'barcodeFile', 'tenXResultDir']:
    cellRangerParamsDf[column] = resultDir + 'step1_cellRanger/' + cellRangerParamsDf[column]
----------------
IN RULE
----------------
rule cellRanger:
    input:
        fastq = lambda wildcard: cellRangerParamsDf.at[wildcard.sample, '

In [35]:
parseUmiDrRule = jps.SnakeRule(snakeFile, 'parseUmiDr', 12)
parseUmiDrRule.addCode("""
parseUmiDrParamsDf = pd.DataFrame(index=sampleLs)
parseUmiDrParamsDf = parseUmiDrParamsDf.assign(output = lambda df:df.index + '_umiDr.tsv', bed = config['genomeBed'])
""")
parseUmiDrRule.addMetaDf('parseUmiDrParamsDf', ['output'])
parseUmiDrRule.addMain('params', ['bamFile', 'barcodeFile'], fromRule=cellRangerRule)
parseUmiDrRule.addMain('params', ['output', 'bed'])
parseUmiDrRule.setShell("""

python ~/scripts/tools/singleCell/parseUmiDirectionFromCellrangerBam.py \
    -i {params.bamFile} -o {params.output} -t {threads} --bed {params.bed} --barcode {params.barcodeFile}
""")
parseUmiDrRule

2022-10-19 16:26:13.874 | INFO     | jpy_tools.parseSnake2:addRule:55 - parseUmiDr step num: 2
2022-10-19 16:26:13.876 | WARNING  | jpy_tools.parseSnake2:addMetaDf:203 - please set `metaDf` if you want to record dataframe content in snakefile



## get parameter of rule `parseUmiDr` ##
parseUmiDrParamsDf = pd.DataFrame(index=sampleLs)
parseUmiDrParamsDf = parseUmiDrParamsDf.assign(output = lambda df:df.index + '_umiDr.tsv', bed = config['genomeBed'])
for column in ['output']:
    parseUmiDrParamsDf[column] = resultDir + 'step2_parseUmiDr/' + parseUmiDrParamsDf[column]
----------------
IN RULE
----------------
rule parseUmiDr:
    input:
        cellRangerFinished = resultDir + 'step1_cellRanger/' + '{sample}.finished',
    output:
        parseUmiDrFinished = resultDir + 'step2_parseUmiDr/' + '{sample}.finished',
    params:
        gpu = 0,
        bamFile = lambda wildcard: cellRangerParamsDf.at[wildcard.sample, 'bamFile'],
        barcodeFile = lambda wildcard: cellRangerParamsDf.at[wildcard.sample, 'barcodeFile'],
        output = lambda wildcard: parseUmiDrParamsDf.at[wildcard.sample, 'output'],
        bed = lambda wildcard: parseUmiDrParamsDf.at[wildcard.sample, 'bed'],
    threads:12
    priority:0
    shell:
        

In [36]:
velocytoRule = jps.SnakeRule(snakeFile, 'velocyto', 12)
velocytoRule.addCode("""
velocytoParamsDf = pd.DataFrame(index=sampleLs)
velocytoParamsDf = velocytoParamsDf.assign(gtf = config['cellRangerIndex'].rstrip('/') + '/genes/genes.gtf')
""")
velocytoRule.addMetaDf('velocytoParamsDf')
velocytoRule.addMain('params', ['tenXResultDir'], fromRule=cellRangerRule)
velocytoRule.addMain('params', ['gtf'])
velocytoRule.setShell("""
velocyto run10x -@ {threads} --samtools-memory 2048 {params.tenXResultDir} {params.gtf}
""")
velocytoRule

2022-10-19 16:26:14.042 | INFO     | jpy_tools.parseSnake2:addRule:55 - velocyto step num: 3
2022-10-19 16:26:14.043 | WARNING  | jpy_tools.parseSnake2:addMetaDf:203 - please set `metaDf` if you want to record dataframe content in snakefile



## get parameter of rule `velocyto` ##
velocytoParamsDf = pd.DataFrame(index=sampleLs)
velocytoParamsDf = velocytoParamsDf.assign(gtf = config['cellRangerIndex'].rstrip('/') + '/genes/genes.gtf')
----------------
IN RULE
----------------
rule velocyto:
    input:
        cellRangerFinished = resultDir + 'step1_cellRanger/' + '{sample}.finished',
    output:
        velocytoFinished = resultDir + 'step3_velocyto/' + '{sample}.finished',
    params:
        gpu = 0,
        tenXResultDir = lambda wildcard: cellRangerParamsDf.at[wildcard.sample, 'tenXResultDir'],
        gtf = lambda wildcard: velocytoParamsDf.at[wildcard.sample, 'gtf'],
    threads:12
    priority:0
    shell:
        """
velocyto run10x -@ {threads} --samtools-memory 2048 {params.tenXResultDir} {params.gtf}
touch {output.velocytoFinished}
        """

In [37]:
velocytoParamsDf = pd.DataFrame(index=sampleLs)
velocytoParamsDf.assign(gtf = config['cellRangerIndex'].rstrip('/') + '/genes/genes.gtf')

,gtf
artis_A3,/public/home/liuzj/data/singleCellRef/mm10_UCS...
artis_A4,/public/home/liuzj/data/singleCellRef/mm10_UCS...


In [38]:
snakeAll = jps.SnakeAll(snakeFile, parseUmiDrRule, velocytoRule)
snakeAll

rule all:
    input:
        parseUmiDrFinished = [resultDir + 'step2_parseUmiDr/' + "" + sample + ".finished" for sample in parseUmiDrParamsDf.index],
        velocytoFinished = [resultDir + 'step3_velocyto/' + "" + sample + ".finished" for sample in velocytoParamsDf.index],

In [39]:
snakeFile.getMain(snakemakeFilePath)

import pandas as pd
#configfile: "/public/home/liuzj/scripts/pipeline/10xAnalysis/snakemake/config.yaml"
pipelineDir = config['pipelineDir']
resultDir = config["resultDir"].rstrip("/") + "/"
pipelineDir = config["pipelineDir"].rstrip("/") + "/"
sampleLs = list(config['Samples'].keys())

## get parameter of rule `cellRanger` ##
cellRangerParamsDt = config["Samples"]
cellRangerParamsDf = pd.DataFrame(cellRangerParamsDt).T.pipe(
    lambda df: df.assign(id=df.index, outputDir=df.index)
)
cellRangerParamsDf = cellRangerParamsDf.pipe(
    lambda df: df.assign(
        bamFile = df.outputDir+ "/" + df.index  + "/outs/possorted_genome_bam.bam",
        barcodeFile = df.outputDir+ "/" + df.index  + "/outs/filtered_feature_bc_matrix/barcodes.tsv.gz",
        tenXResultDir = df.outputDir+ "/" + df.index
    )
)
cellRangerParamsDf["cellRangerIndex"] = config["cellRangerIndex"]
if 'sample' in cellRangerParamsDf.columns:
    pass
else:
    cellRangerParamsDf['sample'] = False
for column in ['output